In [ ]:

import numpy as np 
from tkinter import *
from PIL import Image, ImageFilter
from PIL import ImageTk as itk
import time
import random

# Parameters 

# Grid size 
N = 16
# Resolution 
n = 500

forest_grid = np.zeros((N,N))                  
forest = np.zeros((N,N,3))    
fire_count = 0 

# Growth parameter 
p = 0.01 
# Lightning probability 
f = 0.2  
time_steps = 100

# Create the window for animation
window = Toplevel()

window.geometry(str(int(n*1.2)) + "x" + str(int(n*1.2)))
window.configure(background = 'white')

canvas = Canvas(window, width=500, height=500, bd=2)  
canvas.pack()          

fire_sizes = []
# A list of tuples with coordinates
coordinates = [] 

for time_step in range(time_steps):

    # Reset old burned trees 
    forest_grid[forest_grid == 2] = 0  

    max_cluster = 0
    max_neighbours = 0
    if time_step > 0:

        # Grow trees 
        for x in range(1, N, 1):
            for y in range(1, N, 1):
                
                # Random number for tree generation
                r = round(np.random.uniform(),2) 
                
                coordinate_tuple = (x, y)

                # Grow tree 
                if r <= p and coordinate_tuple not in coordinates and coordinate_tuple: 
                    coordinates.append(coordinate_tuple)
                    forest_grid[x, y] = 1
        
        # Random tuple for lightning strike 
        lightning_coordinates = (random.randint(1, N), random.randint(1, N)) 

        # Remove tree and neighbours if lightning strikes
        if lightning_coordinates in coordinates: 
            coordinates.remove(lightning_coordinates)
            x_lightning, y_lightning = lightning_coordinates
            forest_grid[x_lightning, y_lightning] = 2

            # Get all the negihbours coordinates
            neighbour_tuples = [(x_lightning-1, y_lightning), (x_lightning+1, y_lightning), (x_lightning, y_lightning-1), (x_lightning, y_lightning+1)] 

            # See if the neighbours are a tree, not empty and delete the neighbours 
            neighbour_in_coordinates = [elem for elem in neighbour_tuples if elem in coordinates] 
            new_coordinates = [elem for elem in coordinates if elem not in neighbour_in_coordinates]  
            max_neighbours = len(neighbour_in_coordinates)
           
            all_burned_neighbours = neighbour_in_coordinates

            # Get the neighbours neighbours 
            while len(neighbour_in_coordinates) != 0: 
                x_neighbour, y_neighbour = neighbour_in_coordinates[0]
                neighbour_tuples_new = [(x_neighbour-1, y_neighbour), (x_neighbour+1, y_neighbour), (x_neighbour, y_neighbour-1), (x_neighbour, y_neighbour+1)]
                neighbours_neighbour_in_coord = [elem for elem in neighbour_tuples_new if elem in new_coordinates]
                
                # Delete the negihbours neighbours
                new_coordinates = [elem for elem in new_coordinates if elem not in neighbours_neighbour_in_coord] 
                neighbour_in_coordinates += neighbours_neighbour_in_coord
                
                all_burned_neighbours = all_burned_neighbours + [elem for elem in neighbour_in_coordinates if elem not in all_burned_neighbours]
                max_neighbours += len(neighbours_neighbour_in_coord)

                del neighbour_in_coordinates[0]
            
            for coord in (all_burned_neighbours):
                forest_grid[coord] = 2

            fire_sizes.append(max_neighbours+1)   
            if max_neighbours+1 > max_cluster:
                max_cluster = max_neighbours+1
                
            if max_cluster == 0: 
                max_cluster = 1 

            coordinates = new_coordinates

        # Assign colors 
        forest[:,:,:] = 0                              
        forest[:,:,0] = (forest_grid == 2)*255  
        forest[:,:,1] = (forest_grid == 1)*255  

        forest_image = Image.fromarray(np.uint8(forest), 'RGB').resize((n,n), Image.NEAREST)
        image_sharp = forest_image.filter(ImageFilter.SHARPEN)
        img_sharp2 = itk.PhotoImage(image_sharp)
        img = itk.PhotoImage(forest_image)
        canvas.create_image(0, 0, anchor = NW, image = img) 
        window.title(f"Forest fire with N = {N} at time step {time_step}")

        window.update()

        # If there is a fire burning, wait 
        if sum(sum(forest_grid == 2)) > 0:
            time.sleep(0.01)
        
    
window.quit()
#window.destroy() 

: 